# **Swyp Teddy Bear Club NFT Rewards Calculation**

## **1. Overview**

An **Initial NFT Offering (INO)** is a fundraising method in the cryptocurrency and blockchain ecosystem, specifically tailored for NFTs (Non-Fungible Tokens). Similar to an Initial Coin Offering (ICO), INOs allow creators, projects, or platforms to raise funds by selling a collection of NFTs to early adopters and investors.

In this context, the **Teddy Bear Club NFTs** served as **TeddySwap's NFT offering**. By minting these NFTs, owners earn rewards in **TEDY Tokens**, with the amount determined by both the rarity and quantity of the NFTs they hold. The INO was conducted in two rounds:
- [**Round 1**](https://medium.com/@TeddySwapDEX/teddy-bears-club-minting-utility-and-launch-date-23e9a4e4446d) launched on **February 17, 2023**
- [**Round 2**](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f) launched on **March 8, 2023**

### *Note!*
In TeddySwap's [**initial tokenomics**](https://medium.com/@TeddySwapDEX/teddyswap-tokenomics-fcd700aea552), the market cap was set at **5,000,000,000 TEDY**. However, it was later adjusted to **8,000,000 TEDY** in a [**second announcement**](https://medium.com/@TeddySwapDEX/tedy-tokenomics-v2-fc92723ead0c). This notebook utilizes the original market cap for the calculations, scaling it to the new market cap in the final step.


### **1.1 Purpose**

This document outlines the process for calculating the NFTs' corresponding TEDY token rewards for the Teddy Bear Club NFTs across Round 1 and Round 2. Since the ranking and reward structures differ between the two rounds, separate functions calculate rewards for each round. They are then summed to determine the total rewards, where applicable. 

This document aims to provide a comprehensive explanation of how the token rewards are computed, ensuring transparency and reproducibility for NFT holders.

Feel free to download this notebook and explore it at your convenience.

---

### **1.2 NFT Information**

### *Note!*

Throughout the TEDY life cycle and the duration of the INO, several announcements were released that affected the total TEDY rewards initially determined. Unfortunately, the official Teddy Bears Club can no longer be accessed. The link to those are from the web archive. Below is a summary of these announcements:

1. According to the official [Teddy Bears Club](https://webcf.waybackmachine.org/web/20240905102756/https://tbc.teddyswap.org/) website, the TEDY earnings were updated to include an additional **10% bonus** to the initial values decided in the [initial announcement](https://medium.com/@TeddySwapDEX/teddy-bears-club-minting-utility-and-launch-date-23e9a4e4446d). 

Additionally, as **804** Round 1 NFTs were sold, the tiers were adjusted based on the [source code](https://github.com/teddy-swap/tbc-homepage/blob/main/src/App.tsx) of the official Teddy Bear Club website.

### Round 1

- **Policy ID:** `ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87`
- **Total NFTs:** 804
- **Tier System:**

| **Tier** | **Initial Rarity Range** | **Initial TEDY Allocation** | **Updated Rarity Range** | **Updated TEDY Allocation** |
|----------|--------------------------|-----------------------------|----------------------|-------------------|
| Tier 1   | Rarity 1 - 100     | 28,000  | Rarity 1 - 15    | 30,800                                  |
| Tier 2   | Rarity 101 - 999   | 17,500  | Rarity 16 - 100  | 19,250                                  |
| Tier 3   | Rarity 1,000 - 2,999  | 12,600  | Rarity 101 - 250 | 13,860                                  |
| Tier 4   | Rarity 3,000 - 5,999  | 11,200  | Rarity 251 - 500 | 12,320                                  |
| Tier 5   | Rarity 6,000 - 10,000  | 10,500  | Rarity 501 - 804 | 11,550                                  |

2. The [**initial announcement**](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f) regarding **Round 2** detailed tiers for ranks **1 - 9,200**. Since **2,035** NFTs were sold, the ranks were adjusted accordingly in this [**Discord announcement**](https://discord.com/channels/1053191459597201448/1053207517154508870/1095880926086570105) on the Discord server.

### Round 2

- **Policy ID:** `da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c`
- **Total NFTs:** 2,035
- **Tier System:**

| **Tier** | **Initial Rarity Range** | **Updated Rarity Range** | **Previous MarketCap TEDY** |
|----------|-------------------|------------------------------|---------------|
| Tier 1   | Rarity 1 - 100       |  Rarity 1 - 23             |       7,000         |
| Tier 2   | Rarity 101 - 999     |  Rarity 24 - 221           |      6,000          |
| Tier 3   | Rarity 1,000 - 2,999 |  Rarity 222 - 664          |        5,000        |
| Tier 4   | Rarity 3,000 - 5,999 |  Rarity 665 - 1,327        |        4,500        |
| Tier 5   | Rarity 6,000 - 9,200 |  Rarity 1,328 - 2,035      |         4,200       |

3. Additionally, the [Round 2 announcement](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f) also established that unsold NFTs from the INO would contribute to a surplus reward pool. These TEDY rewards are evenly distributed to NFT holders in proportion to the number of NFTs they own.
---

### **1.3 Workflow Summary**
The workflow is divided into the following steps:

1. **Fetch Data**  
   1.1 Load [`collection_1.json`](./data/collection_1.json) and [`collection_2.json`](./data/collection_2.json) data which was scraped and saved in [`webscraper.ipynb`](./webscraper.ipynb), as well the [`collectiontraits_1.json`](./data/collectiontraits_1.json) and [`collectiontraits_2.json`](./data/collectiontraits_2.json) computed and saved in [`attributecounter.ipynb`](attributecounter.ipynb).

2. **NFT Rank Calculation**  
   2.1 Assign a score to each trait value based on its rarity. <br>
   2.2 Compute the rarity rank for each NFT.

3. **NFT Rewards Calculation**  
   3.1 Load rank data from the [`collectionranks.csv`](./data/collectionranks.csv) created from [`rankdata.ipynb`](rankdata.ipynb)<br>
   3.2 Given a subject, identify its TEDY reward based on its round and rarity rank. <br>
   3.3 Calculate the NFT holder's TEDY allocation of the unsold TEDY tokens.<br>
   3.4 Given an address, calculate the total TEDY rewards owed - utilizing the (3.2)identified tier and (3.3)surplus allocation calculated in previous steps. <br>

4. **Scale Rewards to New Market Cap**<br>
   4.1 Adjust the final TEDY token rewards from the initial [**5,000,000,000**](https://medium.com/@TeddySwapDEX/teddyswap-tokenomics-fcd700aea552) to the updated market cap of [**8,000,000**](https://medium.com/@TeddySwapDEX/tedy-tokenomics-v2-fc92723ead0c).
   
---

### **1.4 Necessary Package Installations and Open Statements**

The following external libraries are required:
- `Plotly.NET`
- `System.Text.Json`
- `FSharp.Data`


In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json"
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"

#r "nuget: Plotly.NET.Interactive, 5.0.0"
#r "nuget: Plotly.NET.ImageExport"

In [75]:
open System.Text.Json
open System.IO
open System.Net.Http
open System.Threading.Tasks
open FSharp.Data
open FSharp.Reflection
open Plotly.NET
open Plotly.NET.Interactive
open Plotly.NET.ImageExport

# **2. Workflow**

## 1. Fetch Data
Metadata for the NFTs was scraped from the official Teddy Bears Club website ([Teddy Bears Club](https://webcf.waybackmachine.org/web/20240905102756/https://tbc.teddyswap.org/)) and [CNFT Tools](https://cnft.tools/teddybearclub2), and stored in [`collection_1.json`](./data/collection_1.json) and [`collection_2.json`](./data/collection_2.json) for Round 1 and Round 2 respectively. Please refer to the [`webscraper.ipynb`](./webscraper.ipynb) to view the process - which you may test out for yourself. 

Additionally, refer to [`attributecounter.ipynb`](./attributecounter.ipynb) for the tallying of trait value occurences, saved in [`collectiontraits_1.json`](./data/collectiontraits_1.json) and [`collectiontraits_2.json`](./data/collectiontraits_2.json).

However, the necessary metadata and trait occurrence data for the NFTs are already included in this repository as JSON files. Therefore, you may directly utilize the above mentioned files without any additional setup.

In this section, we will parse the JSON data to extract information about the NFTs and the occurrence of their traits. This data will serve as the foundation for calculating rankings and determining rewards in the subsequent sections. Using the data from the JSON files, we compute for the NFTs' rarity score and rarity rank. 

---

#### **Types**:
The NFT and NFTTraits types are partial representations of the data, as only the fields relevant to ranking and reward calculations have been included.

- **`NFT`**: Represents an individual NFT, focusing on its identifiers and traits.

- **`NftTraits`**: Captures occurrence counts of trait values across NFTs.

#### **Function**:
1. **`loadNFTs<'T>`**: 
- Loads and deserializes JSON data into a specified type:
- **Parameters**: 
     - `filePath (string)`: the path to the JSON file.
- **Returns**:
    - Parsed data of type `'T`

--- 

#### **Variables**:<br>
**`nftRound1NFTS`**: <br>
- (NFT list): NFTs from Round 1 <br>

 **`nftRound2NFTS`**:  <br>
- (NFT list):  NFTs from Round 2 <br>

 **`nftRound1Traits`**: <br>
- (NFTTraits): Trait value counts for Round 1 NFTs <br>
    
 **`nftRound2Traits`**:  <br>
- (NFTTraits): Trait value counts for Round 2 NFTs<br>

---

### 1.1 Fetch NFT Data From JSON

In [81]:
type NFT = {
    assetName: string
    encodedName: string
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

type NftTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
}

In [5]:
let loadNfts<'T> (filePath: string) : 'T =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<'T>(jsonContent)

In [6]:
let nftRound1NFTS = loadNfts<NFT list> "./data/collection_1.json"
let nftRound2NFTS = loadNfts<NFT list> "./data/collection_2.json"

In [7]:
let nftRound1Traits = loadNfts<NFTTraits> "./data/collectiontraits_1.json"
let nftRound2Traits = loadNfts<NFTTraits> "./data/collectiontraits_2.json"

## **2. NFT Rank Calculation** 
Each NFT has traits such as Background, Bear, Clothing, and more, with each trait having assigned specific values. For example, the Background trait can have values like Space, Lighthouse, or Sunflowers. The overall rarity score and rank of an NFT are determined by the rarity of its trait values.

The rarity score of an NFT is calculated by evaluating the rarity of its trait values. For each trait value, we calculate a **trait score** by determining the percentage of NFTs sharing that value (trait frequency) and subtracting this percentage from 100. This ensures that rarer traits receive higher scores.

Finally, the **total rarity score** is obtained by summing the scores across all trait categories (e.g., Background, Hair, Clothes). The score is rounded to two decimal places for consistency.

$\text{Trait Score} = \text{round}\left(100.0 - \left(\frac{\text{Trait Frequency}}{\text{Total NFTs}}\right) \cdot 100.0, 2\right)$

$\text{Total Rarity Score} = \sum_{i=1}^{7} TraitScore_i$

---
### 2.1 Assign Point Values Based on Trait Value Rarity   
Using the **Trait Score** formula, we compute a score for each trait value. These scores are then stored in a map, organized by trait categories and their respective values. 

---

#### **Function**:
1. **`processNftTraits`**: 
- Calculates rarity scores for each trait value based on the trait data and total NFTs in a round.
- **Parameters**:<br>
    - **`nftTraits`**: (NftTraits): Contains the trait value occurrence data for a round.
        
    - **`totalNfts`**: (float): The total number of NFTs in the round.
- **Returns**: <br>
    - A Map<string, Map<string, float>>, where the outer map corresponds to trait categories, and the inner map holds the scores for each trait value.

---

#### **Variables**:
**`r1TotalNfts`**: 
- (float): Total NFTs in Round 1 (804.0).<br>

**`r2TotalNfts`**: 
- (float): Total NFTs in Round 2 (2035.0).<br>

**`r1ScoredTraits`**: 
- (Map<string, Map<string, float>>): A map of scored trait values for Round 1 NFTs.<br>

**`r2ScoredTraits`**: 
- (Map<string, Map<string, float>>): A map of scored trait values for Round 2 NFTs.<br>


In [8]:
let r1TotalNfts = 804.0
let r2TotalNfts = 2035.0

In [82]:
let processNftTraits (nftTraits: NftTraits) (totalNfts: float) =
    let recordType = typeof<NFTTraits>
    FSharp.Reflection.FSharpType.GetRecordFields(recordType)
    |> Array.fold (fun acc field ->
        let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>
        let traitScores = 
            fieldValue |> Map.map (fun key value ->
                Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2)
            )
        Map.add field.Name traitScores acc
    ) Map.empty

In [80]:
let r1ScoredTraits = processNftTraits nftRound1Traits r1TotalNfts
let r2ScoredTraits = processNftTraits nftRound2Traits r2TotalNfts

### **Illustrating the Distribution of Trait Scores**: 
This section generates visual representations of the distribution of trait scores. The `createCharts` function produces doughnut charts that display each trait value along with its corresponding score.

In [11]:
let createCharts (traitScores: Map<string, Map<string, float>>) = 
    traitScores
    |> Map.fold( fun acc title labelsAndValues ->
        let labels,values = 
            labelsAndValues
            |> Map.toList
            |> List.unzip

        let combinedText = 
            List.map2 (fun label value -> sprintf "%s: %.2f" label value) labels values

        let chart = 
            Chart.Doughnut(values = values, Labels = labels, Hole = 0.5, MultiText = combinedText)
            |> Chart.withTitle title
            |> Chart.withLayout(
                Layout.init(
                    Width = 1200,
                    Height = 1000
                )
            )
                    
        chart :: acc
    )[]
    |> List.rev

let chartRound1 = createCharts r1ScoredTraits 
let chartRound2 = createCharts r2ScoredTraits

### Teddy Bear Club Round 1 Trait Value Scores

<img src="./assets/charts1/doughnut1.png" alt="Round 1 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts1/doughnut2.png" alt="Round 1 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts1/doughnut3.png" alt="Round 1 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts1/doughnut4.png" alt="Round 1 Doughnut Charts" width="1400" height="800">

In [ ]:
for chart in chartRound1 do
    display(chart |> Chart.withSize(1400,1400))

### Teddy Bears Club Round 2 Trait Value Scores

<img src="./assets/charts2/doughnut1.png" alt="Round 2 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts2/doughnut2.png" alt="Round 2 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts2/doughnut3.png" alt="Round 2 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts2/doughnut4.png" alt="Round 2 Doughnut Charts" width="1400" height="800">

In [ ]:
for chart in chartRound2 do
    display(chart |> Chart.withSize(1400,1400)) 

### 2.2 Compute Overall Rarity Ranks for Each NFT. 
This section calculates the total rarity score and rank for each NFT within their respective rounds. The **total rarity score** is determined by summing the scores of all trait values associated with an NFT.

$\text{Total Rarity Score} = \sum_{i=1}^{7} TraitScore_i$

Using the results from this step, a CSV file - [`collectionranks.csv`](./data/collectionranks.csv) - was created to map NFTs to their respective ranks, enabling easier access and computation. This file is referenced in subsequent sections for calculating TEDY token rewards. For more details on generating the CSV, refer to [`rankdata.ipynb`](./rankdata.ipynb).

---

#### **Function**:
1. **`computeNFTRank`**: 
- Computes the rarity rank of NFTs based on their traits and scores.
- **Parameters**:
    - **`nftList`**: (NFT list): The list of NFTs to rank.
    - **`nftTraitScores`**: (Map<string, Map<string, float>>): Trait value scores computed above.

---

#### **Variables**:
**`rankedNFTsRound1`**: 
- (list): Stores the ranked Round 1 NFTs, including their `assetName`, `encodedName`, and `rarityScore`.

**`rankedNFTsRound2`**: 
- (list): Stores the ranked Round 2 NFTs with the same structure as above.


In [14]:
let computeNFTRank (nftList: list<NFT>) nftTraitScores = 
    nftList
    |> List.map (fun nft ->
        let rarityScore =  
            [
                ("Background", nft.Background)
                ("Bear", nft.Bear)
                ("Clothes", nft.Clothes)
                ("Face", nft.Face)
                ("Handheld", nft.Handheld)
                ("Head", nft.Head)
                ("Skins", nft.Skins)
            ]
            |> List.fold (fun acc (attrName, attrValue) ->  
                match Map.tryFind attrName nftTraitScores with
                | Some traitMap ->  
                    acc + (Map.tryFind attrValue traitMap |> Option.defaultValue 0.0)
                | None -> acc 
            ) 0.0 
        (nft.assetName, nft.encodedName, rarityScore) 
    )
    |> List.sortBy (fun (_, _, rarityScore) -> -rarityScore)
    |> List.mapi (fun index (assetName, encodedName, rarityScore) -> 
    (index + 1, assetName, encodedName, rarityScore))

In [15]:
let rankedNFTsRound1 = computeNFTRank nftRound1NFTS r1ScoredTraits
let rankedNFTsRound2 = computeNFTRank nftRound2NFTS r2ScoredTraits

## **3. NFT Rewards Calculation** 
Calculating an NFT holder's TEDY rewards involves multiple factors, including the NFTs they own, the round each NFT belongs to, and their individual ranks within those rounds. 

Additionally, the calculation considers the holder's proportional share of unsold NFTs, based on the number of NFTs they hold.

---
### **3.1 Load rank data from the [`collectionranks.csv`](./data/collectionranks.csv)**
Load NFT rank data from the [`collectionranks.csv`](./data/collectionranks.csv) file in the `data` folder. The CSV was created by using the rank computations executed below. For detailed steps or to recreate the file, refer to [`rankData.ipynb`](./rankdata.ipynb).

#### **Type**:
- **`NftRecord`**: Represents an individual NFT record from the CSV.

#### **Functions**:

1. **`parseCsv`**:
- Parses a CSV file into a list of `NftRecord` entries.
- **Parameters**:
   - `filePath (string)`: The path to the CSV file.
- **Returns**:  
   - A `list<NFTRecord>` containing records with `Name`, `Subject`, `Rank`, and `Score`.

2. **`createSubjectToRankMap`**:
- Creates a mapping of `Subject`(Policy ID + Asset Name) to `Rank`.
- **Parameters**:
   - `records (list<NFTRecord>)`: A list of `NftRecord` entries.
- **Returns**:  
   - A `Map<string, int>` mapping `Subject` to `Rank`.

3. **`createNameToRankMap`**:
- Creates a mapping of `Name` (i.e. TeddyBearsClub178) to `Rank`.
- **Parameters**:
   - `records (list<NFTRecord>)`: A list of `NftRecord` entries.
- **Returns**:  
   - A `Map<string, int>` mapping `Name` to `Rank`.

---

#### **Variables**:

1. **`csvPath`**:  
   - `(string)`: The file path to the CSV file containing NFT rank data.

2. **`nftDataList`**:  
   - `(list<NFTRecord>)`: A list of parsed `NftRecord` entries from the CSV file.

3. **`subjectRankData`**:  
   - `(Map<string, int>)`: A mapping of `Subject` to `Rank` created using `createSubjectToRankMap`.

4. **`nameRankData`**:  
   - `(Map<string, int>)`: A mapping of `Name` to `Rank` created using `createNameToRankMap`.


In [83]:
type NftRecord = {
    Name: string
    Subject: string
    Rank: int
    Score: float
}

In [84]:
 let parseCsv (filePath: string) =
    File.ReadAllLines(filePath)
    |> Array.skip 1
    |> Array.map (fun line ->
        let parts = line.Split(',')
        {
            Name = parts.[0]
            Subject = parts.[1]
            Rank = int parts.[2]
            Score = float parts.[3]
        }
    )
    |> List.ofArray

In [87]:
let createSubjectToRankMap (records: NftRecord list) =
    records
    |> List.map (fun record -> record.Subject, record.Rank)
    |> Map.ofList

let createNameToRankMap (records: NftRecord list) =
    records
    |> List.map (fun record -> record.Name, record.Rank)
    |> Map.ofList

In [88]:
let csvPath = "./data/collectionranks.csv"
let nftDataList = parseCsv csvPath
let subjectRankData = createSubjectToRankMap  nftDataList 
let nameRankData = createNameToRankMap  nftDataList 

### **3.2 Identify a Subject's Tier**
This section categorizes an NFT (identified by its `policyId` and `assetName`) into its respective tier based on its round and rank. Each tier determines the base TEDY rewards allocated to the NFT.

#### **Functions**:
1. **`categorizeNftRound1ByTier`**:
- Categorizes a Round 1 NFT into its respective tier based on its rank.
- **Parameters**:
  - `subject` (string): The identifier of the NFT.
  - `rank` (int): The rarity rank of the NFT.

2. **`categorizeNftRound2ByTier`**:
- Categorizes a Round 2 NFT into its respective tier based on its rank.
- **Parameters**:
  - `subject` (string): The identifier of the NFT.
  - `rank` (int): The rarity rank of the NFT.

3. **`computeBaseRewards`**:
- Determines the base TEDY rewards for a given NFT based on its round and rank.
- Calls `categorizeNftRound1ByTier` or `categorizeNftRound2ByTier` depending on the NFT's policy ID.
- **Parameters**:
  - `subject` (string): The identifier of the NFT.
  - `rankData` (Map<string, int>): A mapping of subjects to their rank.
- **Returns**:
  - `baseReward` (int): The rewards allocated to the given subject.
---

#### **Variables**:
 **`round1PolicyId`**:
   - (string): the unique policy ID for identifying NFTs from Round 1.

 **`round2PolicyId`**:
   - (string): the unique policy ID for identifying NFTs from Round 2.

**`baseReward`**:
   - (int): the amount of TEDY rewards allocated to the NFT.

In [23]:
let round1PolicyId = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87"
let round2PolicyId = "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c"

In [24]:
let categorizeNftRound1ByTier subject rank  =
    if rank <= 15 then 38000
    elif rank <= 100 then 19250
    elif rank <= 250 then 13860
    elif rank <= 500 then 12320
    elif rank <= 804 then 11550
    else 0  

In [25]:
let categorizeNftRound2ByTier subject rank =
    if rank <= 23 then 7000
    elif rank <= 221 then 6000
    elif rank <= 664 then 5000
    elif rank <= 1327 then 4500
    elif rank <= 2035 then 4200
    else 0

In [26]:
let computeBaseRewards (subject: string) (rankData: Map<string, int>) =
    let baseReward =
        if subject.Contains(round1PolicyId) then
            categorizeNftRound1ByTier subject subjectRankData[subject] 
        elif subject.Contains(round2PolicyId) then
            categorizeNftRound2ByTier subject subjectRankData[subject]
        else
            printfn "Unknown NFT policyId for %s" subject
            0
    baseReward

### Computing an NFT's Rank and Reward with Alice’s Teddy Bear

Now that we've built the tools to calculate NFT rankings and determine TEDY rewards, let’s bring these functions to life with a practical example.

Meet Alice, a proud owner of a unique NFT: Teddy Bears Club #5731. Naturally, Alice is curious to find out the rank and TEDY rewards allocated to her her beloved Teddy Bear. Using the tools and functions we’ve developed, we can help Alice uncover this information.

Later on, we will also calculate for the total TEDY tokens Alice can receive. Let’s dive in!

<img src="./assets/teddy5731.png" alt="Teddy Bears Club #5731" width="400" height="400"/>

Let’s start by identifying Alice’s NFT in the system. Alice provides her NFT's subject identifier:


In [76]:
let ownedNFT = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c756235373331"

We can then use the `computeBaseRewards` function to determine the NFT's tier and TEDY rewards. Here’s how we process Alice’s NFT:

In [77]:
let baseReward =  computeBaseRewards ownedNFT subjectRankData

In [78]:
printfn "Alice's Asset: %s" ownedNFT
printfn "Asset's Rank: %d" subjectRankData[ownedNFT]
printfn "Asset's TEDY Reward: %d" baseReward

Alice's Asset: ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c756235373331
Asset's Rank: 162
Asset's TEDY Reward: 13860


### **3.3 Calculate Portion of TEDY Rewards from Surplus TEDY**

In the [Round 2 announcement](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f),it was stated that NFT holders would receive an additional allocation of TEDY tokens from the pool of unsold NFTs. This section explains the formula used to calculate the surplus TEDY rewards allocated per NFT.

Where:

- **Total INO TEDY Allocation:**  
  The total amount of TEDY designated for Initial NFT Offering (INO) rewards based on the [initial tokenomics](https://medium.com/@TeddySwapDEX/teddyswap-tokenomics-fcd700aea552).  
  $\text{Total INO TEDY Allocation} = 5,000,000,000 \times 4.5\% = 225,000,000 \text{TEDY}$
  
- **Round 1 Bonus:**  
  The bonus multiplier applied to NFTs sold in Round 1.  
 $\text{Round 1 Bonus} = 2.75$

- **Round 1 NFTs Sold:**  
  The total number of NFTs sold in Round 1.  
  $\text{Round 1 NFTs Sold} = 804$

- **Round 2 NFTs Sold:**  
  The total number of NFTs sold in Round 2.  
  $\text{Round 2 NFTs Sold} = 2,035$

- $\text{Tier Tokens}_i$:  
  The number of TEDY tokens allocated per NFT in Tier \(i\).

- $\text{Count of Tier}_i$:
  The number of NFTs in Tier \(i\).

#### **Calculations**

1. **Calculate Distributed Tokens:**

   Determine the total TEDY tokens already distributed based on the tiers of NFTs sold in both rounds.

   $\text{Distributed Tokens} = \sum_{i=1}^{5} (\text{Round 1 Tier Tokens}_i \times \text{Count of Tier}_i) + \sum_{i=1}^{5} (\text{Round 2 Tier Tokens}_i \times \text{Count of Tier}_i)$

2. **Determine Available Token Rewards:**

   Calculate the remaining TEDY tokens available for surplus distribution by subtracting the distributed tokens from the total INO TEDY allocation.

   $\text{Available Token Rewards} = \text{Total INO TEDY Allocation} - \text{Distributed Tokens}$

3. **Compute TEDY Surplus per NFT:**

   Calculate the additional TEDY reward allocated per NFT by dividing the available token rewards by the effective total number of NFTs, considering the bonus multiplier for Round 1 NFTs.

   $\text{TEDY Surplus per NFT} = \frac{\text{Available Token Rewards}}{(\text{Round 1 NFTs Sold} \times \text{Round 1 Bonus}) + \text{Round 2 NFTs Sold}}$

---

#### **Functions**:
1. **`calculateSurplusTedyPerNFT`**:
- Computes the additional TEDY rewards allocated per NFT owned by a holder.
- **Parameters**:
   - None explicitly passed, but it relies on:
      - `availableTokenRewards` (float) - The total amount of TEDY rewards available.
      - `round1Bonus` (float) - The bonus multiplier for Round 1 NFTs.
      - `round1NFTsSold` (int) - Total number of NFTs sold in Round 1.
      - `round2NFTsSold` (int)  Total number of NFTs sold in Round 2.
- **Returns**:
   - (float) - The calculated TEDY surplus reward per NFT, rounded up to the nearest whole number.

---

#### **Variables**:
**`round1Bonus`**:
   - (float): represents the bonus multiplier applied to Round 1 NFTs.

**`round1NFTsSold`**:
   - (int): represents the total NFTs minted for Round 1.

**`round2NFTsSold`**:
   - (int): represents the total NFTs minted for Round 2.
   <br>

**`round1Tier1`, `round1Tier2`, `round1Tier3`, `round1Tier4`, `round1Tier5`**:
   - Tokens allocated per NFT for each tier in Round 1:
     - **round1Tier1**: `30800`
     - **round1Tier2**: `19250`
     - **round1Tier3**: `13860`
     - **round1Tier4**: `12320`
     - **round1Tier5**: `11550`

**`round2Tier1`, `round2Tier2`, `round2Tier3`, `round2Tier4`, `round2Tier5`**:
   - Tokens allocated per NFT for each tier in Round 2:
     - **round2Tier1**: `7000`
     - **round2Tier2**: `6000`
     - **round2Tier3**: `5000`
     - **round2Tier4**: `4500`
     - **round2Tier5**: `4200`

**`round1CountOfTier1`, `round1CountOfTier2`, `round1CountOfTier3`, `round1CountOfTier4`, `round1CountOfTier5`**:
   - The number of NFTs held in each tier:
     - **countOfTier1**: `15`
     - **countOfTier2**: `85`
     - **countOfTier3**: `150`
     - **countOfTier4**: `250`
     - **countOfTier5**: `304`

**`round2CountOfTier1`, `round2CountOfTier2`, `round2CountOfTier3`, `round2CountOfTier4`, `round2CountOfTier5`**:
   - The number of NFTs held in each tier:
     - **countOfTier1**: `23`
     - **countOfTier2**: `198`
     - **countOfTier3**: `443`
     - **countOfTier4**: `663`
     - **countOfTier5**: `708`

**`distributedTokens`**:
   - (int): The total tokens distributed across all both rounds.

**`availableTokenRewards`**:
   - (int): The remaining TEDY tokens available for INO rewards after distribution.


In [32]:
let round1Bonus = 2.75
let round1NftsSold = 804.0
let round2NftsSold = 2035.0

In [39]:
let round1Tier1 = 30800
let round1Tier2 = 19250
let round1Tier3 = 13860
let round1Tier4 = 12320
let round1Tier5 = 11550

let round1CountOfTier1 = 15
let round1CountOfTier2 = 85
let round1CountOfTier3 = 150
let round1CountOfTier4 = 250
let round1CountOfTier5 = 304

let round2Tier1 = 7000
let round2Tier2 = 6000
let round2Tier3 = 5000
let round2Tier4 = 4500
let round2Tier5 = 4200

let round2CountOfTier1 = 23
let round2CountOfTier2 = 198
let round2CountOfTier3 = 443
let round2CountOfTier4 = 663
let round2CountOfTier5 = 708

let totalTokenRewards = 225000000

In [40]:
let round1Counts = [| round1CountOfTier1; round1CountOfTier2; round1CountOfTier3; round1CountOfTier4; round1CountOfTier5 |]
let round1Tiers = [| round1Tier1; round1Tier2; round1Tier3; round1Tier4; round1Tier5 |]
let round2Counts = [| round2CountOfTier1; round2CountOfTier2; round2CountOfTier3; round2CountOfTier4; round2CountOfTier5 |]
let round2Tiers = [| round2Tier1; round2Tier2; round2Tier3; round2Tier4; round2Tier5 |]

In [41]:
let distributedTokens = 
    (Array.zip round1Counts round1Tiers |> Array.sumBy (fun (count, tier) ->  count *  tier)) +
    (Array.zip round2Counts round2Tiers |> Array.sumBy (fun (count, tier) ->  count *  tier))


In [42]:
let availableTokenRewards =  totalTokenRewards - distributedTokens

In [44]:
let calculateSurplusTedyPerNFT = 
    let surplusReward = float availableTokenRewards/(round1NftsSold * round1Bonus + round2NftsSold)
    Math.Ceiling(surplusReward)

In [46]:
let surplusReward = int calculateSurplusTedyPerNFT

## **3.4 Calculate Total TEDY Rewards** 
In this step, we calculate the total TEDY rewards for a given address based on the NFTs it holds.

This process involves parsing the address's token data from `collectionassets.json`. 

To begin, you must **generate the JSON** file containing the address's token data by following the workflow outlined in [`api.ipynb`](./api.ipynb). This file, `collectionassets.json`, is created by querying the [CardanoScan API](https://docs.cardanoscan.io/). The JSON file is not included in this repository, and you must generate it yourself if you wish to explore these calculations.

Once the token data is available, we use the `computeBaseRewards` function and include the `surplusReward` to compute the total TEDY rewards for the address.

*Note!*
You will need to replace the placeholder with the address you wish to query.

---

#### **Types**:
- **`Token`**: Represents a token/NFT, focusing on its identifiers and balance.
- **`Response`**: Represents the response structure containing a list of tokens.
  - `tokens (Token[])`: An array of `Token` objects.

---

#### **Function**:
1. **`calculateTotalRewards`**:
   - Computes the total TEDY rewards based on the Teddy Bear Club NFTs owned by an address.
   - **Parameters**:
     - `filePath (string)`: The path to the JSON file containing token data.
     - `round1PolicyId (string)`: The policy ID for tokens in Round 1.
     - `round2PolicyId (string)`: The policy ID for tokens in Round 2.
     - `subjectRankData (Map<string, int>)`: A mapping of token IDs to their respective ranks.
     - `surplusReward (int)`: Additional reward to be distributed across all tokens.
   - **Returns**:
     - `totalTedyRewards (int)`: The total calculated rewards.

---

#### **Variable**:
- **`totalTedyRewards`**: 
  - (int) — Total TEDY rewards allocated to the address, calculated as the sum of base rewards for all NFTs and the surplus reward distributed across all tokens. This calculation assumes the original 5,000,000,000 market cap.


In [48]:
type Token = {
    policyId: string
    assetName: string
    fingerprint: string
    assetId: string
    balance: string
}

type Response = {
    tokens: Token[]
}

In [57]:
let calculateTotalRewards (filePath: string) (round1PolicyId: string) (round2PolicyId: string) (subjectRankData: Map<string, int>) (surplusReward: float) =
    let addressNfts = loadNfts<Response> filePath

    let nftList =
        addressNfts.tokens
        |> Array.map (fun token -> token.assetId)
        |> Array.toList

    let rewardsList =
        [ for nft in nftList do
            if nft.Contains(round1PolicyId) then
                yield computeBaseRewards nft subjectRankData
            elif nft.Contains(round2PolicyId) then
                yield computeBaseRewards nft subjectRankData ]

    let totalNftRewards = rewardsList |> List.sum
    let numberOfNfts = rewardsList |> List.length
    let totalTedyRewards = totalNftRewards + ((int)surplusReward * numberOfNfts)

    totalTedyRewards


In [55]:
let totalTedyRewards = calculateTotalRewards "./data/collectionassets.json" round1PolicyId round2PolicyId subjectRankData surplusReward
printfn "Total Rewards of Address: %d" totalTedyRewards


Total Rewards of Address: 52413


### **4. Adjust Rewards for Market Cap**
In this step, we adjust the final TEDY reward allocation to reflect the updated market cap of 8,000,000, ensuring alignment with the revised tokenomics. The adjustment scales the rewards originally calculated for the initial market cap of 5,000,000,000.

Where:
- Total TEDY Allocation: The total rewards calculated based upon the initial marketcap.
- [Current Marketcap](https://medium.com/@TeddySwapDEX/tedy-tokenomics-v2-fc92723ead0c): 8,000,000
- [Previous Market Cap](https://medium.com/@TeddySwapDEX/teddyswap-tokenomics-fcd700aea552): 5,000,000

$\text{Final Round Reward} = (\frac{\text{Total TEDY Allocation} * \text{Current Marketcap}}{\text{Previous Market Cap}})$

---

#### **Functions**:
1. **`marketcapAdjustment`**:
- Adjusts the reward amount based on the ratio of the current market cap to the past market cap.
- **Parameters**:
   - `rewards` (float): The original reward amount to be adjusted.
- **Returns**:
   - (int): The adjusted reward amount.

---

#### **Variables**:
**`pastMarketCap`**:
   - The previous market cap of the token, used as a reference for adjustments.

**`currentMarketCap`**:
   - The current market cap of the token, used to scale rewards.

**`totalTedyReward`**:
   - The final TEDY rewards allocation 


In [68]:
let initialMarketCap = 5000000000.0
let currentMarketCap = 8000000.0

In [90]:
let marketcapAdjustment (rewards: float) = 
    let finalRoundReward = (rewards*currentMarketCap)/initialMarketCap
    Math.Ceiling(finalRoundReward)

In [89]:
let totalTedyReward = int(marketcapAdjustment totalTedyRewards)

# Alice's Total TEDY Rewards

Now that we’ve developed the functions to calculate rewards, it’s time to discover the total TEDY rewards Alice will receive. Unfortunately, we don't have Alice's address, so we will be running through the calculations manually - to provide an example to follow.

We saw above that as a Round 1 NFT, ranked 162 and placed in Tier 3, #5731 earns Alice a total of **13,860 TEDY tokens**. Furthermore, she gains a surplus reward of 48,213 * 1 = **48,213**, having only one asset. 

Her total reward is the sum of the two, 13,860 + 48,213 = **62,073**.

Finally, we scale her rewards according to the new market cap of 8,000,000.

$\text{Final Round Reward} = (\frac{\text{62,073} * \text{8,000,000}}{\text{5,000,000,000}})$

In [71]:
let totalReward = baseReward + surplusReward
let finalReward = Math.Ceiling((float totalReward * currentMarketCap) / initialMarketCap)


In [73]:
printfn "#5731 Base Reward: %d" baseReward
printfn "Alice's Surplus Reward: %d" surplusReward
printfn "Alice's Initial TEDY Rewards: %d" totalReward
printfn "Alice's Total TEDY Rewards: %d" (int finalReward)

#5731 Base Reward: 13860
Alice's Surplus Reward: 48213
Alice's Initial TEDY Rewards: 62073
Alice's Total TEDY Rewards: 100


## **Disclaimers**
1. *Market Cap Adjustments*:
The calculations above are based on the initial market cap of **5,000,000,000** used to model events during the INO. `marketcapAdjustment` scales rewards to align with the updated market cap of **8,000,000**, resulting in adjusted and lower TEDY reward amounts.

The figures provided are *estimates* and are intended for informational purposes only. We do not guarantee their accuracy and disclaim liability for any investment decisions made based on this information. Use this data at your own risk.

2. Exclusions:
These NFT calculations **do not** account for the following bonuses promised by TeddySwap:

- FISO (Fair Initial Stake Offering) bonuses
- ITN (Incentivized Testnet) rewards
- LBE (Liquidity Bootstrapping Event) rewards
- Yield farming bonuses
